In [36]:
import pandas as pd
import numpy as np
import re

In [15]:
data = pd.read_csv('amazon_baby.csv')
print(data.shape)
data.head()

(183531, 3)


,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


In [23]:
data.dropna(inplace=True)
data = data[data['rating'] != 3]
print(data.shape)

(165679, 14)


In [24]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 
                  'terrible', 'awful', 'wow', 'hate']
for w in selected_words:
    data[w] = data['review'].apply(lambda x: 
                    list(map(str.lower, re.split('\W+', x))).count(w))

In [25]:
data[selected_words].sum()

awesome       3881
great        55677
fantastic     1661
amazing       2624
love         41918
horrible      1109
bad           4181
terrible      1145
awful          687
wow            425
hate          1107
dtype: int64

In [26]:
data['sentiment'] = (data['rating'] > 3).astype('int')

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
train, test = train_test_split(data, train_size=0.8, random_state=0)
selected_words_model = LogisticRegression(C=1e6)
selected_words_model.fit(X=train[selected_words], y=train['sentiment'])

LogisticRegression(C=1000000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [31]:
for w, c in zip(selected_words, selected_words_model.coef_[0]):
    print(w, c)

awesome 1.24199125224
great 0.870528979156
fantastic 0.94844553562
amazing 1.08459117009
love 1.38994008605
horrible -2.27444667236
bad -0.974476801992
terrible -2.15283450126
awful -1.98728414899
wow -0.182715761477
hate -1.3825393481


In [37]:
np.mean(selected_words_model.predict(test[selected_words]) == test['sentiment'])

0.84940849830999521

In [38]:
np.mean(test['sentiment'])

0.8424070497344278

In [43]:
np.mean(train[train['name']=='Baby Trend Diaper Champ']['sentiment'])

0.8565400843881856

In [48]:
selected_words_model.predict_proba(
    test[test['name'] == 'Baby Trend Diaper Champ'][selected_words])[:,1].max()

0.98373712918302314

In [51]:
train[train['name']=='Baby Trend Diaper Champ'][selected_words].sum()

awesome       2
great        75
fantastic     3
amazing       2
love         79
horrible      3
bad          26
terrible      0
awful         1
wow           2
hate          2
dtype: int64